## 数据处理

In [11]:
import pandas as pd
import io
import re
from datetime import datetime, timedelta
from typing import List, Union, Optional
from datetime import datetime

import os, re, json, time
import typing as T
import requests
from pathlib import Path
from openpyxl import Workbook, load_workbook

from openpyxl.utils import get_column_letter
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
from data_processing import load_and_process,build_jsonl_for_range, save_jsonl


In [12]:
## 研发字典
speaker_map = {
    "16186514":   "peter本尊",
    "1655611808": "运营绾绾",
    "2073820674": "沙利文老师",
    "2726067525": "milissa",
}
## 客服字典
MAPPING_FILE = "mapping地球1.xlsx"

##QQ的txt文件
pathtxt   = "1209《欢迎来到地球》测试2群.txt"

# 设定时间范围
start_time = "2025-12-03 00:00:00"
end_time   = "2025-12-04 00:00:00"


# 1) 拿到 JSONL（列表）
jsonl_lines01 = build_jsonl_for_range(
    pathtxt=pathtxt,
    mapping_file=MAPPING_FILE,
    speaker_map=speaker_map,
    start_time=start_time,
    end_time=end_time,
    return_str=False,   # 返回 list[str]
)

print(len(jsonl_lines01), "lines")
print(jsonl_lines01[0])



4555 lines
{"发言日期": "2025-12-03", "发言时间": "00:00:11", "玩家ID": "摆(2970667974)", "玩家消息": "@火山 @火山 生气了"}


## 大模型分类

## 定义

In [13]:
"""
批处理 10000 条聊天数据（每批 100 条）：
- 模型#1：过滤非游戏相关（只保留相关 JSON 行，原样输出）
- 模型#2：提取高讨论的发言并分析
- 结果按word格式文档输出
"""
from model_classifyV1_Copy1_Copy1 import (
    load_system_prompt,
    build_user_prompt_filter,build_user_prompt_clsuter,call_ark_chat_completions,
    extract_valid_json_lines,add_index_to_jsonl_lines,count_output_filter_stats,get_covered_indices_from_cluster_output,
    aggregate_cluster_outputs,build_user_prompt_cluster_agg,assign_global_cluster_ids,
    extract_top5_heat_clusters,attach_discussion_points,extract_cluster_stats,append_daily_top5_to_version_jsonl
)

## 设置参数

In [14]:
# ============= 你的模型与文件配置（改这里） =============
API_URL   = "https://ark.cn-beijing.volces.com/api/v3/chat/completions" 
API_KEY = "de91deb0-aae6-46cb-bac0-17ac3b6107f5" #API
V3_MODEL_ID= "ep-20251020160142-5d7hp"#接入点
V3_1_MODEL_ID = "ep-20251020160025-9p5tj"#接入点
R1_MODEL_ID = "ep-20251020160103-5n6g2"#接入点

PROMPT_MD_PATH01 = Path("提示词1.md") # 模型#1 system 提示词（筛相关）
PROMPT_MD_PATH02 = Path("2话题分类.md") # 模型#2 system 提示词（分话题）
PROMPT_MD_PATH03 = Path("3日聚合.md") # 模型#3 system 提示词（日聚合）
VERSION_TOP5_JSONL = "daily_top5.jsonl"
BATCH_SIZE       = 300
SLEEP_BETWEEN    = 1   # 每批之间的间隔，防止QPS触发限流；按需调整
RETRIES          = 2
TEMPERATURE      = 0.20
MAX_TOKENS       = 16384
TIMEOUT_SEC      = 600
# =====================================================

# 1.检验每日话提簇分类输出

In [15]:
import json
import time
from tqdm import tqdm



# ===========================================================
# =============== 主循环：模型#1 + 模型#2 ====================
# ===========================================================

# 读取系统提示
system_prompt01 = load_system_prompt(PROMPT_MD_PATH01)  # 筛相关
system_prompt02 = load_system_prompt(PROMPT_MD_PATH02)  # 做话题簇
# 1⃣ 给原始 jsonl 每条加 _idx
jsonl_lines01_indexed = add_index_to_jsonl_lines(jsonl_lines01)

total = len(jsonl_lines01_indexed)

PRINT_UNCLUSTERED = False
if total == 0:
    print("没有可处理的数据。")
else:
    total_batches = (total + BATCH_SIZE - 1) // BATCH_SIZE
    print(f"准备处理 {total} 条，共 {total_batches} 批（每批 {BATCH_SIZE} 条）。")

# 统计总量用的（可选）
total_filtered_lines = 0         # 模型#1 输出总行数之和（含客服）
total_filtered_player_lines = 0  # 模型#1 输出玩家行数之和
total_covered_idx = 0            # 被话题簇覆盖的原始发言总数（按 _idx 去重）


for b in tqdm(range(total_batches), desc="🔥 批处理进度", unit="批"):
    start = b * BATCH_SIZE
    end = min(start + BATCH_SIZE, total)
    # 本批原始发言（已带 _idx）
    batch_lines = jsonl_lines01_indexed[start:end]

    # 本批原始 _idx 集合，用来和话题簇覆盖做差集
    batch_original_idx = set()
    for line in batch_lines:
        try:
            obj = json.loads(line)
            batch_original_idx.add(int(obj["_idx"]))
        except Exception:
            continue

    try:
        # ========== 模型 #1：筛相关 ==========
        user_prompt1 = build_user_prompt_filter(batch_lines)
        output_filter = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_MODEL_ID,
            system_prompt=system_prompt01,
            user_prompt=user_prompt1,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        if not output_filter:
            tqdm.write(f"[批次 {b+1}] 模型#1 无有效输出，跳过。")
            continue

        # 统计筛相关之后的行数：总行数 & 玩家行数
        filter_total_lines, filter_player_lines = count_output_filter_stats(output_filter)
        total_filtered_lines += filter_total_lines
        total_filtered_player_lines += filter_player_lines

        tqdm.write(
            f"[批次 {b+1}] 模型#1 输出总行数：{filter_total_lines}，"
            f"其中玩家发言：{filter_player_lines} 条（不去重）"
        )

        # ========== 模型 #2：话题簇 ==========
        user_prompt2 = build_user_prompt_clsuter(output_filter)
        output_cluster = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,
            system_prompt=system_prompt02,
            user_prompt=user_prompt2,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        if not output_cluster:
            tqdm.write(f"[批次 {b+1}] 模型#2 无有效输出，跳过。")
            continue

        # 打印本批的话题簇输出（原样）
        print(f"\n===== 批次 {b+1}/{total_batches} 话题簇输出 =====")
        print(output_cluster)
        print("===========================================\n")

        # 计算：本批有哪些 _idx 被话题簇覆盖（从“发言行号列表”解析）
        covered_idx = get_covered_indices_from_cluster_output(output_cluster)
        total_covered_idx += len(covered_idx)

        # 和“本批原始发言 _idx 集合”做差集，看有哪些没被覆盖
        unclustered_idx = batch_original_idx - covered_idx

        tqdm.write(
            f"[批次 {b+1}] 话题簇覆盖原始发言条数：{len(covered_idx)}，"
            f"未被覆盖条数：{len(unclustered_idx)}"
        )

        # ⭐⭐ 新增：打印“未被话题簇覆盖”的发言信息（最多前 20 条）
        if PRINT_UNCLUSTERED and unclustered_idx:
            # 先把这一批中未覆盖的发言对象捞出来
            unclustered_records = []
            for line in batch_lines:
                try:
                    obj = json.loads(line)
                except Exception:
                    continue
                idx_val = obj.get("_idx")
                try:
                    idx_int = int(idx_val)
                except Exception:
                    continue
                if idx_int in unclustered_idx:
                    unclustered_records.append(obj)

            # 按 _idx 排序，避免乱序
            unclustered_records.sort(key=lambda x: int(x.get("_idx", 0)))

            print(f"\n[批次 {b+1}] 未被话题簇覆盖的原始发言（最多前20条）：")
            for rec in unclustered_records[:20]:
                idx = rec.get("_idx")
                date = rec.get("发言日期") or rec.get("日期") or ""
                t = rec.get("发言时间") or rec.get("时间") or ""
                speaker = rec.get("发言人ID") or rec.get("玩家ID") or rec.get("角色ID") or ""
                msg = (
                    rec.get("玩家消息")
                    or rec.get("发言内容")
                    or rec.get("玩家发言")
                    or rec.get("消息")
                    or ""
                )
                print(f"- _idx={idx} [{date} {t}] {speaker}: {msg}")
            print()  # 换行分隔一下

    except Exception as e:
        tqdm.write(f"[批次 {b+1}] ❌ 出错：{e}")
        continue

    # 防止 QPS 过高
    time.sleep(SLEEP_BETWEEN)

print("\n✅ 全部批次处理完成！")
print(f"原始输入总数：{total}")
print(f"模型#1 筛相关后总行数：{total_filtered_lines}，其中玩家发言：{total_filtered_player_lines} 条")
print(f"模型#2 话题簇累计覆盖原始发言条数：{total_covered_idx}")

准备处理 141 条，共 1 批（每批 300 条）。


🔥 批处理进度:   0%|                                                                                                                                     | 0/1 [02:39<?, ?批/s]      

[批次 1] 模型#1 输出总行数：89，其中玩家发言：89 条（不去重）


🔥 批处理进度:   0%|                                                                                                                                     | 0/1 [02:46<?, ?批/s]      


===== 批次 1/1 话题簇输出 =====
{"话题簇":"停服维护补偿讨论（2025-11-19 15:02:03-15:11:27）","核心对象/机制":"停服维护补偿机制，玩家讨论维护时长与补偿期望（100抽、50抽、石币、代币等）"}
{"话题簇":"游戏下载与安装问题（2025-11-19 15:02:22-15:13:20）","核心对象/机制":"游戏安装包下载与更新机制，玩家询问下载链接、是否需要重新下载等问题"}
{"话题簇":"角色技能更新讨论（2025-11-19 15:02:58-15:04:02）","核心对象/机制":"猛犸象和仙人掌角色的PTSD与强迫症被动技能更新实装情况"}
{"话题簇":"客服互动与调侃（2025-11-19 15:07:27-15:14:50）","核心对象/机制":"客服服务体验，玩家与客服薏米、蓝桉的互动调侃及无关游戏内容的闲聊"}

[批次 1] 话题簇覆盖原始发言条数：0，未被覆盖条数：141


🔥 批处理进度: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [02:47<00:00, 167.86s/批]


✅ 全部批次处理完成！
原始输入总数：141
模型#1 筛相关后总行数：89，其中玩家发言：89 条
模型#2 话题簇累计覆盖原始发言条数：0


## 开始运行

In [ ]:
from tqdm import tqdm
import time
import json
from pathlib import Path

# ... 前置：系统提示、create_intent_excel_styled(EXCEL_FILE)、jsonl_lines01 等

system_prompt01 = load_system_prompt(PROMPT_MD_PATH01)  # 筛相关
system_prompt02 = load_system_prompt(PROMPT_MD_PATH02)  # 做话提簇
system_prompt03 = load_system_prompt(PROMPT_MD_PATH03)  # 话题簇聚合 / 校正（智能体4）

# 用于存放当天所有批次的话题簇 JSON 行（给智能体4用）
batch_cluster_outputs  = []
total = len(jsonl_lines01)
if total == 0:
    print("没有可处理的数据。")
else:
    total_batches = (total + BATCH_SIZE - 1) // BATCH_SIZE
    print(f"准备处理 {total} 条，共 {total_batches} 批（每批 {BATCH_SIZE} 条）。")

written_total = 0

for b in tqdm(range(total_batches), desc="🔥 批处理进度", unit="批"):
    start = b * BATCH_SIZE
    end = min(start + BATCH_SIZE, total)
    batch_lines = jsonl_lines01[start:end]

    try:
        # --- 模型 #1：筛相关 ---
        user_prompt1 = build_user_prompt_filter(batch_lines)
        output_filter = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_MODEL_ID,
            system_prompt=system_prompt01,
            user_prompt=user_prompt1,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        
        if not output_filter:
            tqdm.write(f"[批次 {b+1}] 模型#1 无有效输出，跳过。")
            continue
        filter_count = sum(1 for line in output_filter.splitlines() if line.strip())
        tqdm.write(f"[批次 {b+1}] 模型#1 筛后保留 {filter_count} 条")
        written_total += filter_count   # 如果不需要总数，可以删掉这一行
        
        # --- 模型 #2：话题簇划分 ---
        user_prompt2 = build_user_prompt_clsuter(output_filter)
        output_cluster = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,
            system_prompt=system_prompt02,
            user_prompt=user_prompt2,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        
        

        if not output_cluster:
            tqdm.write(f"[批次 {b+1}] 模型#2 无有效输出，跳过。")
            continue
        try:
            cluster_json_list = []
            for line in output_cluster.strip().splitlines():
                line = line.strip()
                if not line.startswith("{"):
                    continue
                obj = json.loads(line)
                # 将 "话题簇1" → "话题簇"，便于后续统一处理
                for key in list(obj.keys()):
                    if key.startswith("话题簇") and key != "话题簇":
                        obj["话题簇"] = obj.pop(key)
                cluster_json_list.append(obj)

            # 假设从输出中拿日期
            date_str = "2025-11-15"  # 你可以从上下文获得更准确值
            batch_id = f"B{b+1}"
            cluster_json_list = assign_global_cluster_ids(cluster_json_list, date_str, batch_id)
        
            output_cluster_with_ids = "\n".join(json.dumps(c, ensure_ascii=False) for c in cluster_json_list)
            batch_cluster_outputs.append(output_cluster_with_ids)
        except Exception as e:
            tqdm.write(f"[批次 {b+1}] ⚠ 添加 _cluster_id 失败：{e}")
            continue

       
       
    except Exception as e:
        tqdm.write(f"[批次 {b+1}] ❌ 出错：{e}")
        continue

    time.sleep(SLEEP_BETWEEN)



# --- 每日话提簇聚合成jsonl ---
all_cluster = aggregate_cluster_outputs(batch_cluster_outputs)

# --- 模型 #3：日话提簇聚合 ---
user_prompt3 = build_user_prompt_cluster_agg(all_cluster)
output_cluster_agg = call_ark_chat_completions(
    api_url=API_URL,
    api_key=API_KEY,
    model=V3_1_MODEL_ID,           # 或你给智能体4选的模型
    system_prompt=system_prompt03,  # 话题簇聚合/校正提示词
    user_prompt=user_prompt3,
    temperature=TEMPERATURE,
    max_tokens=MAX_TOKENS,
    timeout=TIMEOUT_SEC,
    retries=RETRIES,
)

parsed_subclusters = [
    json.loads(line.strip()) for line in all_cluster.strip().splitlines() if line.strip()
]
parsed_clusters = [json.loads(line) for line in output_cluster_agg.strip().splitlines() if line.strip()]
top5_results = extract_top5_heat_clusters(parsed_clusters, jsonl_lines01, top_k=5)
final_result = attach_discussion_points(top5_results, parsed_subclusters)

# 输出查看：
#for row in final_result:
    #print(json.dumps(row, ensure_ascii=False, indent=2))

append_daily_top5_to_version_jsonl(
    final_result,
    version_jsonl_path=VERSION_TOP5_JSONL # 这个路径你可以按版本号动态改
)



    
    

准备处理 4555 条，共 16 批（每批 300 条）。


🔥 批处理进度:   0%|                                                                                                                                    | 0/16 [00:00<?, ?批/s]

In [6]:
print(output_cluster)

{"话题簇": "特殊重启契约书获取与区别讨论（2025-11-20 22:30:26-22:31:28）", "核心对象/机制": "重启契约书系统的特殊类型获取方式与功能区别探讨"}
{"话题簇": "已出战装甲更换机制咨询（2025-11-20 23:28:27-23:37:50）", "核心对象/机制": "装甲系统出战后的更换限制与重启重置机制"}
{"话题簇": "地图图腾缺失问题反馈（2025-11-20 23:49:32-23:54:55）", "核心对象/机制": "地图系统中图腾奖励缺失的bug反馈与重启解决方案"}
